In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
subject = 'Piauí - Consumo de Cimento (t)'
split_index = 191 #Referente aos 230 anos de input  
train_split = split_index + 1 -30

In [6]:
data = pd.read_csv('2003_mo_model_input_PI.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Piauí - IDH,Piauí - Produção de Cimento (t),Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - Desemprego,Piauí - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.649218,24.669760,2.136063e+07,1.358044e+06,6.061141,1.868149e+07,8.192266,21.844
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.649661,25.167145,2.139102e+07,1.359835e+06,6.065065,1.869506e+07,8.186156,15.814
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.650104,25.520404,2.142142e+07,1.361626e+06,6.068988,1.870864e+07,8.180046,16.067
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.650548,25.519725,2.145181e+07,1.363417e+06,6.072912,1.872222e+07,8.173937,14.938
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.650991,25.929312,2.148221e+07,1.365208e+06,6.076835,1.873580e+07,8.167827,17.765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.957215,NaN,NaN,NaN,NaN,NaN,77.776
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.965575,NaN,NaN,NaN,NaN,NaN,72.377
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.977734,NaN,NaN,NaN,NaN,NaN,71.791
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.994123,NaN,NaN,NaN,NaN,NaN,71.015


In [7]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Piauí - IDH,Piauí - Produção de Cimento (t),Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - Desemprego
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.255392,-1.240720,-1.669677,-2.498915,-2.014630,-1.966414,-0.747080
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.215344,-1.193299,-1.652630,-2.431501,-1.986722,-1.939126,-0.751564
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.175296,-1.159620,-1.635584,-2.364087,-1.958813,-1.911838,-0.756047
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-2.135248,-1.159684,-1.618537,-2.296673,-1.930905,-1.884550,-0.760531
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-2.095200,-1.120635,-1.601491,-2.229259,-1.902997,-1.857261,-0.765014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.336332,-1.934378,1.184371,0.511529,1.001536,1.022413,1.331534
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.329819,-1.960422,1.174437,0.526506,0.987984,1.008864,1.330230
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.323306,-1.985417,1.164504,0.541484,0.974431,0.995315,1.328926
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.316792,-2.009327,1.154570,0.556461,0.960879,0.981765,1.327622


In [8]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      18.413
1      16.605
2      22.084
3      22.445
4      23.323
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Piauí - Consumo de Cimento (t), Length: 240, dtype: float64

In [9]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Piauí - IDH,Piauí - Produção de Cimento (t),Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - Desemprego
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.255392,-1.240720,-1.669677,-2.498915,-2.014630,-1.966414,-0.747080
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.215344,-1.193299,-1.652630,-2.431501,-1.986722,-1.939126,-0.751564
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.175296,-1.159620,-1.635584,-2.364087,-1.958813,-1.911838,-0.756047
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-2.135248,-1.159684,-1.618537,-2.296673,-1.930905,-1.884550,-0.760531
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-2.095200,-1.120635,-1.601491,-2.229259,-1.902997,-1.857261,-0.765014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.382405,-0.210180,1.192759,0.339852,1.041980,1.070621,1.460989
158,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.385453,-0.282985,1.199072,0.340094,1.049362,1.077532,1.459107
159,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.388501,-0.356527,1.205385,0.340336,1.056744,1.084444,1.457225
160,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.391549,-0.430817,1.211698,0.340578,1.064126,1.091356,1.455343


In [10]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0      18.413
1      16.605
2      22.084
3      22.445
4      23.323
        ...  
157    54.656
158    69.984
159    58.643
160    72.337
161    70.968
Name: Piauí - Consumo de Cimento (t), Length: 162, dtype: float64

In [11]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Piauí - IDH,Piauí - Produção de Cimento (t),Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - Desemprego
162,-0.601510,-0.653307,0.157879,-1.255326,1.088045,0.763466,-1.213929,1.397646,-0.581677,1.224324,0.341061,1.078891,1.105179,1.451580
163,-0.786068,-0.653814,0.548855,-1.118679,1.099377,0.752299,-1.292173,1.400694,-0.658262,1.230638,0.341303,1.086273,1.112091,1.449698
164,-0.830387,-0.645656,0.385685,-0.936336,1.110709,0.741131,-1.324219,1.403742,-0.735635,1.236951,0.341545,1.093655,1.119003,1.447816
165,-0.801089,-0.634274,0.420965,-0.931790,1.122042,0.729964,-1.344446,1.406790,-0.813815,1.243264,0.341787,1.101037,1.125915,1.445934
166,-0.959917,-0.624035,0.548870,-1.168522,1.133374,0.718796,-1.381638,1.409838,-0.892819,1.249577,0.342029,1.108419,1.132826,1.444052
167,-1.022309,-0.619791,0.222499,-1.285611,1.144706,0.707629,-1.411208,1.412886,-0.972668,1.255890,0.342271,1.115801,1.139738,1.442171
168,-1.074401,-0.620112,-0.614754,-1.446316,1.156038,0.696461,-1.412953,1.415934,-1.053381,1.262203,0.342513,1.123183,1.146650,1.440289
169,-1.119597,-0.615973,-0.478229,-1.357832,1.167287,0.681823,-1.491464,1.413100,-1.134978,1.261512,0.347861,1.120952,1.144201,1.431987
170,-1.078648,-0.609777,-0.393157,-1.235682,1.178536,0.667184,-1.573805,1.410266,-1.212485,1.260820,0.353209,1.118720,1.141751,1.423685
171,-1.055426,-0.602141,-0.557552,-1.065340,1.189784,0.652545,-1.564950,1.407432,-1.291035,1.260129,0.358557,1.116488,1.139302,1.415383


In [12]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

162    75.913
163    74.774
164    72.484
165    71.534
166    70.320
167    67.364
168    59.688
169    40.259
170    41.239
171    45.035
172    42.477
173    57.619
174    53.109
175    63.141
176    55.580
177    60.001
178    62.767
179    45.308
180    57.142
181    46.283
182    43.390
183    40.824
184    55.063
185    56.474
186    68.133
187    68.642
188    64.741
189    71.269
190    67.631
191    62.845
Name: Piauí - Consumo de Cimento (t), dtype: float64

In [13]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*div_factor + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [14]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 8)
    target,target_val = validation_splitter(train_target, 8)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [17]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [18]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[4121761216, 3905059143, 2654542077, 2470464604, 187176261, 1689011358, 1962526129, 3163657453, 2562722345, 1984944408, 41051988, 2558800775, 1553869388, 528453202, 988644813, 3967487080, 2643060102, 4025605482, 247896754, 556665803, 1776816470, 761443290, 2177346346, 115890705, 3354919160]


Step: 0 ___________________________________________
val_loss: 17.42163848876953
winner_seed: 4121761216


Step: 1 ___________________________________________
val_loss: 20.529523849487305


Step: 2 ___________________________________________
val_loss: 23.375125885009766


Step: 3 ___________________________________________
val_loss: 21.93442153930664


Step: 4 ___________________________________________
val_loss: 22.27463150024414


Step: 5 ___________________________________________
val_loss: 17.776607513427734


Step: 6 ___________________________________________
val_loss: 15.633512496948242
winner_seed: 1962526129


Step: 7 ___________________________________________
val_loss: 24.277324676513672

In [19]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 1s 26ms/step - loss: 2443.0247 - val_loss: 1948.3652
Epoch 2/10000
5/5 [==============================] - 0s 6ms/step - loss: 2028.1067 - val_loss: 2218.2258
Epoch 3/10000
5/5 [==============================] - 0s 6ms/step - loss: 2207.3862 - val_loss: 2763.6919
Epoch 4/10000
5/5 [==============================] - 0s 6ms/step - loss: 2144.8013 - val_loss: 2114.3862
Epoch 5/10000
5/5 [==============================] - 0s 6ms/step - loss: 1475.6202 - val_loss: 401.8625
Epoch 6/10000
5/5 [==============================] - 0s 6ms/step - loss: 907.5742 - val_loss: 261.1328
Epoch 7/10000
5/5 [==============================] - 0s 5ms/step - loss: 1013.1503 - val_loss: 1438.4813
Epoch 8/10000
5/5 [==============================] - 0s 5ms/step - loss: 1440.5437 - val_loss: 1315.3861
Epoch 9/10000
5/5 [==============================] - 0s 5ms/step - loss: 1286.4066 - val_loss: 1110.7117
Epoch 10/10000
5/5 [==============================] - 0s 

5/5 [==============================] - 0s 6ms/step - loss: 101.3006 - val_loss: 81.4722
Epoch 81/10000
5/5 [==============================] - 0s 6ms/step - loss: 103.9620 - val_loss: 81.2988
Epoch 82/10000
5/5 [==============================] - 0s 5ms/step - loss: 108.2661 - val_loss: 101.6977
Epoch 83/10000
5/5 [==============================] - 0s 6ms/step - loss: 111.1714 - val_loss: 85.6862
Epoch 84/10000
5/5 [==============================] - 0s 6ms/step - loss: 106.3868 - val_loss: 82.1080
Epoch 85/10000
5/5 [==============================] - 0s 5ms/step - loss: 104.5496 - val_loss: 89.4799
Epoch 86/10000
5/5 [==============================] - 0s 6ms/step - loss: 105.2650 - val_loss: 81.6692
Epoch 87/10000
5/5 [==============================] - 0s 5ms/step - loss: 103.9098 - val_loss: 91.5014
Epoch 88/10000
5/5 [==============================] - 0s 6ms/step - loss: 108.5627 - val_loss: 85.4096
Epoch 89/10000
5/5 [==============================] - 0s 6ms/step - loss: 109.1305 - va

5/5 [==============================] - 0s 6ms/step - loss: 63.0495 - val_loss: 78.1225
Epoch 160/10000
5/5 [==============================] - 0s 6ms/step - loss: 79.9994 - val_loss: 63.5058
Epoch 161/10000
5/5 [==============================] - 0s 6ms/step - loss: 83.7220 - val_loss: 84.3186
Epoch 162/10000
5/5 [==============================] - 0s 7ms/step - loss: 85.6016 - val_loss: 56.3095
Epoch 163/10000
5/5 [==============================] - 0s 5ms/step - loss: 72.0486 - val_loss: 48.8162
Epoch 164/10000
5/5 [==============================] - 0s 7ms/step - loss: 58.7860 - val_loss: 61.4750
Epoch 165/10000
5/5 [==============================] - 0s 6ms/step - loss: 65.2662 - val_loss: 45.6730
Epoch 166/10000
5/5 [==============================] - 0s 5ms/step - loss: 56.4227 - val_loss: 41.2646
Epoch 167/10000
5/5 [==============================] - 0s 5ms/step - loss: 45.7178 - val_loss: 41.7734
Epoch 168/10000
5/5 [==============================] - 0s 5ms/step - loss: 53.5873 - val_

5/5 [==============================] - 0s 5ms/step - loss: 49.3309 - val_loss: 30.5672
Epoch 239/10000
5/5 [==============================] - 0s 5ms/step - loss: 47.8003 - val_loss: 18.3162
Epoch 240/10000
5/5 [==============================] - 0s 5ms/step - loss: 38.1106 - val_loss: 27.4216
Epoch 241/10000
5/5 [==============================] - 0s 6ms/step - loss: 39.1281 - val_loss: 31.6366
Epoch 242/10000
5/5 [==============================] - 0s 6ms/step - loss: 86.2604 - val_loss: 42.8468
Epoch 243/10000
5/5 [==============================] - 0s 5ms/step - loss: 62.8539 - val_loss: 74.5520
Epoch 244/10000
5/5 [==============================] - 0s 5ms/step - loss: 65.5873 - val_loss: 38.9109
Epoch 245/10000
5/5 [==============================] - 0s 5ms/step - loss: 61.6167 - val_loss: 43.5452
Epoch 246/10000
5/5 [==============================] - 0s 7ms/step - loss: 69.7186 - val_loss: 47.0626
Epoch 247/10000
5/5 [==============================] - 0s 5ms/step - loss: 65.0922 - val_

5/5 [==============================] - 0s 6ms/step - loss: 29.0616 - val_loss: 38.7091
Epoch 318/10000
5/5 [==============================] - 0s 6ms/step - loss: 31.1666 - val_loss: 42.0478
Epoch 319/10000
5/5 [==============================] - 0s 6ms/step - loss: 35.9307 - val_loss: 53.2369
Epoch 320/10000
5/5 [==============================] - 0s 6ms/step - loss: 37.2670 - val_loss: 29.6850
Epoch 321/10000
5/5 [==============================] - 0s 5ms/step - loss: 36.0695 - val_loss: 37.4984
Epoch 322/10000
5/5 [==============================] - 0s 5ms/step - loss: 33.5272 - val_loss: 24.1224
Epoch 323/10000
5/5 [==============================] - 0s 6ms/step - loss: 34.4474 - val_loss: 41.1946
Epoch 324/10000
5/5 [==============================] - 0s 5ms/step - loss: 33.6999 - val_loss: 35.1372
Epoch 325/10000
5/5 [==============================] - 0s 5ms/step - loss: 29.8173 - val_loss: 31.0821
Epoch 326/10000
5/5 [==============================] - 0s 6ms/step - loss: 30.9188 - val_

...

5/5 [==============================] - 0s 6ms/step - loss: 18.2791 - val_loss: 34.3462
Epoch 792/10000
5/5 [==============================] - 0s 6ms/step - loss: 21.2204 - val_loss: 35.8373
Epoch 793/10000
5/5 [==============================] - 0s 6ms/step - loss: 22.2271 - val_loss: 33.9234
Epoch 794/10000
5/5 [==============================] - 0s 7ms/step - loss: 24.1928 - val_loss: 33.0742
Epoch 795/10000
5/5 [==============================] - 0s 5ms/step - loss: 20.0584 - val_loss: 26.9747
Epoch 796/10000
5/5 [==============================] - 0s 5ms/step - loss: 19.4926 - val_loss: 29.8699
Epoch 797/10000
5/5 [==============================] - 0s 6ms/step - loss: 21.7789 - val_loss: 30.2591
Epoch 798/10000
5/5 [==============================] - 0s 6ms/step - loss: 19.7491 - val_loss: 26.7398
Epoch 799/10000
5/5 [==============================] - 0s 6ms/step - loss: 19.9460 - val_loss: 45.7721
Epoch 800/10000
5/5 [==============================] - 0s 6ms/step - loss: 24.3880 - val_

5/5 [==============================] - 0s 6ms/step - loss: 17.1421 - val_loss: 33.0462
Epoch 871/10000
5/5 [==============================] - 0s 5ms/step - loss: 19.5686 - val_loss: 39.3929
Epoch 872/10000
5/5 [==============================] - 0s 5ms/step - loss: 18.2499 - val_loss: 32.2105
Epoch 873/10000
5/5 [==============================] - 0s 5ms/step - loss: 20.8472 - val_loss: 28.1878
Epoch 874/10000
5/5 [==============================] - 0s 5ms/step - loss: 17.6133 - val_loss: 27.3212
Epoch 875/10000
5/5 [==============================] - 0s 5ms/step - loss: 18.1119 - val_loss: 26.9051
Epoch 876/10000
5/5 [==============================] - 0s 6ms/step - loss: 18.1581 - val_loss: 35.2513
Epoch 877/10000
5/5 [==============================] - 0s 5ms/step - loss: 19.7488 - val_loss: 28.7526
Epoch 878/10000
5/5 [==============================] - 0s 6ms/step - loss: 18.5168 - val_loss: 26.9632
Epoch 879/10000
5/5 [==============================] - 0s 5ms/step - loss: 17.5566 - val_

5/5 [==============================] - 0s 5ms/step - loss: 23.9972 - val_loss: 29.6802
Epoch 950/10000
5/5 [==============================] - 0s 5ms/step - loss: 19.1669 - val_loss: 36.0422
Epoch 951/10000
5/5 [==============================] - 0s 5ms/step - loss: 20.5462 - val_loss: 38.3126
Epoch 952/10000
5/5 [==============================] - 0s 5ms/step - loss: 21.3212 - val_loss: 39.2898
Epoch 953/10000
5/5 [==============================] - 0s 5ms/step - loss: 20.2045 - val_loss: 37.2986
Epoch 954/10000
5/5 [==============================] - 0s 5ms/step - loss: 17.4974 - val_loss: 34.3294
Epoch 955/10000
5/5 [==============================] - 0s 5ms/step - loss: 19.3873 - val_loss: 32.9200
Epoch 956/10000
5/5 [==============================] - 0s 5ms/step - loss: 17.2491 - val_loss: 27.2868
Epoch 957/10000
5/5 [==============================] - 0s 6ms/step - loss: 18.9016 - val_loss: 29.3824
Epoch 958/10000
5/5 [==============================] - 0s 6ms/step - loss: 20.0638 - val_

5/5 [==============================] - 0s 5ms/step - loss: 19.4273 - val_loss: 30.5857
Epoch 1029/10000
5/5 [==============================] - 0s 6ms/step - loss: 17.3119 - val_loss: 31.5404
Epoch 1030/10000
5/5 [==============================] - 0s 5ms/step - loss: 17.4695 - val_loss: 30.8389
Epoch 1031/10000
5/5 [==============================] - 0s 5ms/step - loss: 18.2652 - val_loss: 23.5046
Epoch 1032/10000
5/5 [==============================] - 0s 5ms/step - loss: 24.0377 - val_loss: 27.1889
Epoch 1033/10000
5/5 [==============================] - 0s 5ms/step - loss: 20.5365 - val_loss: 33.0482
Epoch 1034/10000
5/5 [==============================] - 0s 7ms/step - loss: 22.4525 - val_loss: 23.9201
Epoch 1035/10000
5/5 [==============================] - 0s 6ms/step - loss: 21.8477 - val_loss: 23.0192
Epoch 1036/10000
5/5 [==============================] - 0s 6ms/step - loss: 21.2743 - val_loss: 28.2571
Epoch 1037/10000
5/5 [==============================] - 0s 6ms/step - loss: 19.58

Epoch 1107/10000
5/5 [==============================] - 0s 8ms/step - loss: 21.6918 - val_loss: 37.1825
Epoch 1107: early stopping


In [20]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [21]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            test_input, 
                                            test_target, 
                                            train_split)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 15ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,69.253922,69.258904,69.254097,69.250778,69.191338,35.17173,24.148249,24.444172,24.509846,23.805075,22.185478,26.600525,22.201387,22.142048,23.984703,24.678234,26.129097,22.140841,22.132366,22.136606,22.137009,22.135777,22.137989,22.135357,22.164333,24.604904,24.584856,24.458918,24.794937,24.627682
Target,75.913,74.774,72.484,71.534,70.32,67.364,59.688,40.259,41.239,45.035,42.477,57.619,53.109,63.141,55.58,60.001,62.767,45.308,57.142,46.283,43.39,40.824,55.063,56.474,68.133,68.642,64.741,71.269,67.631,62.845
Error,6.659081,5.515099,3.229904,2.283218,1.128662,32.192268,35.539749,15.814827,16.729153,21.229925,20.291523,31.018475,30.907614,40.998951,31.595299,35.322765,36.637901,23.167158,35.009632,24.146395,21.252991,18.688225,32.925011,34.338642,45.96867,44.037094,40.156143,46.810081,42.83606,38.217319


In [22]:
display(mae)
display(mape)

27.15493

0.47398442

In [23]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [24]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             train_split)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 16ms/step
Ano-0: |Prediction[[527.07416]] - Target[718.706]| =  Error: [[191.63184]]; MAPE:[[0.26663452]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[274.09143]] - Target[639.082]| =  Error: [[364.99054]]; MAPE:[[0.5711169]]
1/1 [==============================] - 0s 14ms/step
Ano-5: |Prediction[[145.23563]] - Target[403.26099999999997]| =  Error: [[258.02536]]; MAPE:[[0.63984704]]


[array([[191.63184]], dtype=float32),
 array([[364.99054]], dtype=float32),
 array([[258.02536]], dtype=float32)]

271.54922

0.49253282